# **W1D1T1-T3.ipynb**

Author: Zhixian Yang

Email: [yangzhx28@mail2.sysu.edu.cn](mailto:yangzhx28@mail2.sysu.edu.cn) or [yimu01439@gmail.com](yimu01439@gmail.com)

GitHub: [https://github.com/koar-create](https://github.com/koar-create)

Date created: July 17th, 2023

Last modified: July 24th, 2023

<br><br>

---

<br><br>

## **Description**
This document is a Jupyter Notebook designed for an exercise derived from the "Computational Tools for Climate Science 2023" course offered by Climatematch Academy. The code presented here comprises a combination of materials provided in the course and code obtained from online sources.

In [1]:
import urllib.request
from itertools import product
import s3fs, boto3, botocore, pooch
from pythia_datasets import DATASETS
import os, sys, glob, platform, tempfile
from datetime import datetime, timedelta
import numpy as np, pandas as pd, xarray as xr
import matplotlib as mpl, matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs, cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

# **<font color='red' size=7>section 1</font>**

In [2]:
rand_data   = 273.16 + 5 * np.random.randn(5, 20, 50)
times_index = pd.date_range('2018-01-01', periods=5)
lon         = np.linspace(-170, 170, 50)
lat         = np.linspace( -60,  60, 20)
T           = xr.DataArray(rand_data, coords=[times_index, lat, lon], dims=['time', 'lat', 'lon'])
T

<xarray.DataArray (time: 5, lat: 20, lon: 50)>
array([[[277.55477704, 279.30942409, 275.4183813 , ..., 280.80956787,
         275.14933564, 280.82416173],
        [275.71293127, 263.52461443, 273.94545482, ..., 276.04472957,
         279.06996979, 271.05462146],
        [270.3778456 , 281.55809312, 261.51403742, ..., 270.69050098,
         281.0252397 , 271.91150359],
        ...,
        [272.74001015, 270.49323025, 262.37628041, ..., 273.24216293,
         273.67348418, 271.01258335],
        [273.10312197, 267.17551385, 271.56284312, ..., 271.59033327,
         265.74232775, 277.3959104 ],
        [273.17879405, 272.85341103, 270.45367824, ..., 268.3828714 ,
         275.58031142, 270.65793566]],

       [[275.08959554, 269.57889746, 262.68994859, ..., 270.60860988,
         280.06462851, 279.4544162 ],
        [276.70038617, 268.3217658 , 271.76701234, ..., 275.39860894,
         275.91007417, 278.78215296],
        [270.33954289, 277.87810338, 274.02613114, ..., 277.20363423,
         268.49404469, 273.2901078 ],
...
        [276.58475048, 274.4466936 , 269.90412091, ..., 272.38885749,
         269.8581865 , 263.9967933 ],
        [266.80351798, 275.5811267 , 281.38872712, ..., 280.25168362,
         272.6880198 , 268.10503061],
        [280.72826395, 274.58081001, 277.25072768, ..., 270.4754422 ,
         275.14159766, 272.06447477]],

       [[268.91504282, 270.88845404, 272.13674976, ..., 275.60914175,
         281.91160742, 274.96905984],
        [287.68683362, 262.06541528, 277.7000418 , ..., 279.09660698,
         280.65539732, 268.61899433],
        [262.85454628, 268.50965496, 270.32216524, ..., 277.22518287,
         280.42999234, 279.42286843],
        ...,
        [265.70104867, 275.98841778, 275.00201212, ..., 274.94259977,
         273.83846593, 274.19092276],
        [267.23052066, 263.60750469, 264.56169591, ..., 270.19427922,
         284.57596974, 269.76996816],
        [275.23013519, 270.49325854, 264.53607041, ..., 276.17501623,
         280.81153624, 268.48058263]]])
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat      (lat) float64 -60.0 -53.68 -47.37 -41.05 ... 41.05 47.37 53.68 60.0
  * lon      (lon) float64 -170.0 -163.1 -156.1 -149.2 ... 156.1 163.1 170.0

In [3]:
T.attrs['units'] = 'kelvin'
T.attrs['long_name'] = "temperature"

In [4]:
p = 1013.25 + 5 * np.random.randn(5, 20, 50)
p = xr.DataArray(p, coords=[times_index, lat, lon], dims=['time', 'lat', 'lon'])
p.attrs['units'] = "pascal"
p.attrs['long_name'] = "pressure"

# **<font color='red' size=7>section 2</font>**

In [5]:
ds = xr.Dataset({"pressure": p, "temperature": T})
ds

<xarray.Dataset>
Dimensions:      (time: 5, lat: 20, lon: 50)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-01-05
  * lat          (lat) float64 -60.0 -53.68 -47.37 -41.05 ... 47.37 53.68 60.0
  * lon          (lon) float64 -170.0 -163.1 -156.1 -149.2 ... 156.1 163.1 170.0
Data variables:
    pressure     (time, lat, lon) float64 1.012e+03 1.015e+03 ... 1.01e+03
    temperature  (time, lat, lon) float64 277.6 279.3 275.4 ... 280.8 268.5

In [6]:
np.isin(ds.pressure.time.dt.year, [2018])
np.isin(ds.pressure.time.dt.year, [2017, 2019])

array([False, False, False, False, False])

# **<font color='red' size=7>section 3</font>**

In [7]:
ds.pressure.sel(time="2018-01-02")

<xarray.DataArray 'pressure' (lat: 20, lon: 50)>
array([[1022.38872381, 1010.16227601, 1010.04895513, 1014.5983922 ,
        1016.69358408, 1007.16402948, 1018.00642591, 1011.89710579,
        1012.64606212, 1011.23852993, 1005.3225038 , 1010.73646088,
        1005.42063343, 1016.2617643 , 1011.37383485, 1008.56697302,
        1002.35088848, 1016.25185144, 1020.53824064, 1013.55980814,
        1012.55249915, 1019.19395977, 1013.56580358, 1020.21175903,
        1012.6932698 , 1017.32601037, 1018.01987112, 1013.63171065,
        1005.78235554, 1009.49116018, 1012.36875002, 1014.01989338,
        1009.909082  , 1017.71112091, 1011.75189997, 1012.0600897 ,
        1009.86134183, 1018.63468224, 1009.52998271, 1008.922711  ,
        1013.5541334 , 1009.24693248, 1011.69215471, 1011.25647057,
        1012.35912506, 1013.85759575, 1015.41578919, 1016.14796285,
        1014.03215518, 1011.61851978],
       [1025.33803422, 1022.6615845 , 1021.70522934, 1010.46075915,
        1013.68432303, 1005.66088092, 1009.97698413, 1017.19242709,
        1007.10528081, 1008.33500123, 1018.41139999, 1009.31110831,
        1017.59867437, 1019.71422622, 1019.1310765 , 1010.07267231,
        1026.79874157, 1008.52068371, 1012.53292286, 1014.78565929,
        1015.78252204, 1005.3336414 , 1020.67468847, 1008.81356191,
        1004.61633562, 1008.21320697, 1013.7313077 , 1013.43252753,
...
        1014.99548143, 1011.76666369, 1016.15106149, 1004.15183137,
        1010.93553019, 1010.85009646, 1013.88430314, 1013.77515946,
        1010.49862434, 1011.85638687, 1017.10077826, 1007.80515269,
        1015.24078641, 1007.43792174, 1010.08536631, 1014.97680201,
        1007.37364459, 1011.5773795 , 1018.75452208, 1007.71983886,
        1015.79683178, 1013.83999638, 1014.06823109, 1015.86842029,
        1014.0236094 , 1010.24751764],
       [1015.16932115, 1019.83972493, 1013.81022556, 1007.92398467,
        1016.3497592 , 1011.31259137, 1020.42992978, 1019.63082193,
        1011.99609643, 1014.5658569 , 1019.11680065, 1013.56976519,
        1014.78857526, 1011.18729956, 1015.11614485, 1016.62572711,
        1009.42047788, 1007.81372704, 1011.53734611, 1016.58236401,
        1011.3124657 , 1015.35108444, 1014.69058973, 1017.30603731,
        1008.92114079, 1019.61974587, 1015.60837984, 1017.55864114,
        1012.62834448, 1019.47437084, 1016.59551847, 1017.92333923,
        1009.20590647, 1018.80159791, 1012.02184053, 1007.88378015,
        1016.65217916, 1011.63260754, 1009.8065567 , 1010.93147865,
        1015.36902137, 1024.89304901, 1001.53055192, 1021.60790833,
        1018.91305645, 1018.29633898, 1019.03434917, 1010.51321505,
        1004.59030984, 1019.67465252]])
Coordinates:
    time     datetime64[ns] 2018-01-02
  * lat      (lat) float64 -60.0 -53.68 -47.37 -41.05 ... 41.05 47.37 53.68 60.0
  * lon      (lon) float64 -170.0 -163.1 -156.1 -149.2 ... 156.1 163.1 170.0
Attributes:
    units:      pascal
    long_name:  pressure